In [1]:
import numpy as np
import copy

In [2]:
data = np.genfromtxt('day14_input.txt', delimiter='\n', dtype=np.str)

reactions = [] #input at [:][0], output at [:][1]
for j in range(0, len(data)):
    inp, outp = data[j].split('=>')
    inp = inp.split(',')
    for i in range(0, len(inp)):
        inp[i] = inp[i].split()
        inp[i][0] = int(inp[i][0])

    outp = outp.split()
    outp[0] = int(outp[0])

    reactions.append([inp, outp])

In [3]:
def ore_for_fuel(target_fuel, reactions):
    #Find fuel
    for i in range(0, len(reactions)):
        if reactions[i][1][1] == 'FUEL':
            fuel = i
            break

    need = copy.deepcopy(reactions[fuel][0])
    #Multiply FUEL components by amount of fuel wanted
    for i in range(0, len(need)):
        need[i][0] *= target_fuel
    surplus = []

    while True:
        #find reaction
        target = need[0][1]
        for i in range(0, len(reactions)):
            if reactions[i][1][1] == target:
                idx = i
                break

        #find if surplus
        in_surplus = False
        surplus_idx = -1
        extra = 0
        for i in range(0, len(surplus)):
            if surplus[i][1] == target:
                in_surplus = True
                surplus_idx = i
                extra = surplus[i][0]
                break

        if need[0][0] > extra:
            #find number of reactions needed
            target_made = reactions[idx][1][0]
            target_need = need[0][0]-extra
            if in_surplus:
                surplus.pop(surplus_idx)

            mltp = target_need//target_made

            #If excess, store it
            if target_need % target_made != 0:
                mltp += 1
                extra = target_made - (target_need % target_made)
                surplus.append([extra, target])

            #add input to need list
            for i in range(0, len(reactions[idx][0])):
                in_need = False
                for j in range(0, len(need)):
                    if need[j][1] == reactions[idx][0][i][1]:
                        need[j][0] += mltp*reactions[idx][0][i][0]
                        in_need = True
                        break
                if not in_need:
                    need.append([mltp*reactions[idx][0][i][0], reactions[idx][0][i][1]])
        else:
            #Use surplus
            left = extra - need[0][0]
            if left == 0:
                surplus.pop(surplus_idx)
            else:
                surplus[surplus_idx][0] = left

        #remove need[0]
        need.pop(0)
        #Move ORE to last position if in first
        if need[0][1] == 'ORE' and len(need) > 0:
            ore_entry = need[0]
            need.pop(0)
            need.append(ore_entry)

        #Get ORE
        if len(need) == 1 and need[0][1] == 'ORE':
            return need[0][0]

In [4]:
ore_per_fuel = ore_for_fuel(1, reactions)
print('Part 1 Solution:', ore_per_fuel)

Part 1 Solution: 1185296


In [5]:
avaliable_ore = 1000000000000
ore_per_fuel = ore_for_fuel(1, reactions)
min_fuel = avaliable_ore//ore_per_fuel
max_fuel = min_fuel * 2

#Binary Search
while min_fuel <= max_fuel:
    test_fuel = (min_fuel+max_fuel) // 2
    if test_fuel == min_fuel:
        break
    test_ore = ore_for_fuel(test_fuel, reactions)
        
    if test_ore > avaliable_ore:
        max_fuel = test_fuel
    elif test_ore < avaliable_ore:
        min_fuel = test_fuel
    else:
        min_fuel = test_fuel
        break
        
print('Part 2 Solution:', min_fuel)

Part 2 Solution: 1376631
